In [2]:
import pandas as pd
df = pd.read_csv('tweets.csv', encoding='latin1')
print(df.columns.values)

['target' 'id' 'date' 'flag' 'user' 'text']


In [3]:
data = pd.DataFrame()
data['text'] = df.text
del df

In [4]:
tweets = data['text']

In [9]:
print(len(tweets))
print(tweets[0])

1600000
@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D


In [11]:
import nltk as nlp
import re
from nltk.corpus import stopwords

lemma = nlp.WordNetLemmatizer()
text = []
for each in data.text:
    each = re.sub("[^a-zA-Z]", " ", str(each))
    # lowercase all
    each = each.lower()
    # split all by tokenizing
    each = nlp.word_tokenize(each)
    # delete stop words from your array
    each = [word for word in each if not word in set(
        stopwords.words("english"))]
    # lemmatize "memories" -> "memory"
    each = [lemma.lemmatize(word) for word in each]
#     # put them into big array
    text.append(each)

In [1]:
# import pickle
# with open('tweets-lemma.txt', 'wb') as f:
#     pickle.dump(text, f)
# with open('tweets-lemma.txt', 'rb') as f:
    # text = pickle.load(f)

In [1]:
import gensim
from gensim.models import Word2Vec, KeyedVectors
model = Word2Vec(text, min_count=2)
# model.wv.save_word2vec_format('word2vec-tweets')
# model = KeyedVectors.load_word2vec_format('word2vec-tweets')

In [7]:
print(model.similar_by_word('brother'))

[('sister', 0.841433048248291), ('cousin', 0.7108091115951538), ('bro', 0.6551007628440857), ('bros', 0.6453346610069275), ('dad', 0.6419702768325806), ('nephew', 0.637410581111908), ('maggiex', 0.6359193921089172), ('brothes', 0.6213483214378357), ('limor', 0.6047430634498596), ('victoriax', 0.5994175672531128)]


In [ ]:
# Take the 15000 most common words
word_vectors = model.syn0[:15000]
n_words = word_vectors.shape[0]
vector_size = word_vectors.shape[1]
print(n_words, vector_size)

15000 100


In [38]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.15, min_samples=2, n_jobs=4, metric='cosine')
idx = dbscan.fit_predict(word_vectors)

In [39]:
import numpy as np
print('Number of clusters: ', len(np.unique(idx)))

Number of clusters:  429


In [40]:
word_centroid_list = list(zip(model.index2word, idx))
word_centroid_list_sort = sorted(word_centroid_list, key=lambda el: el[1], reverse=False)

In [36]:
cluster = {}
file_out = open('cluster-output', "w")
file_out.write("WORD\tCLUSTER_ID\n")
for word_centroid in word_centroid_list_sort:
    line = word_centroid[0] + '\t' + str(word_centroid[1]) + '\n'
    cluster[word_centroid[0]] = word_centroid[1]
    file_out.write(line)
file_out.close()

In [41]:
# See if there's any semantic meaning captured in the clustering
words = ['coffee', 'tea', 'water','good', 'bad',
         'beautiful', 'ugly', 'pen', 'chocolate',
         'food', 'pizza', 'hungry', 'company', 'water', 'rain', 'sun',
         'hurricane', 'brother', 'sister', 'father', 'niece', 'son',
         'school', 'college', 'university',
         'english', 'french', 'german',
         'hindi', 'tamil',
         ]
for word in words:
    print(word, cluster[word])

coffee 4
tea 4
water -1
good 1
bad -1
beautiful 51
ugly -1
pen -1
chocolate 4
food -1
pizza 4
hungry 83
company -1
water -1
rain 4
sun 4
hurricane -1
brother 4
sister 4
father 67
niece 4
son 109
school 21
college 21
university 4
english 4
french -1
german 4
hindi 4
tamil 4
